# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pokemon = pd.read_csv('Pokemon.csv')
pokemon

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [3]:
# your code here

# Observamos primero la matriz de correlación para los atributos que queremos examinar
cols = pokemon.columns[4:11]
pokemon[cols].corr()

,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
Total,1.000000,0.618748,0.736211,0.612787,0.747250,0.717609,0.575943
HP,0.618748,1.000000,0.422386,0.239622,0.362380,0.378718,0.175952
Attack,0.736211,0.422386,1.000000,0.438687,0.396362,0.263990,0.381240
Defense,0.612787,0.239622,0.438687,1.000000,0.223549,0.510747,0.015227
Sp. Atk,0.747250,0.362380,0.396362,0.223549,1.000000,0.506121,0.473018
Sp. Def,0.717609,0.378718,0.263990,0.510747,0.506121,1.000000,0.259133
Speed,0.575943,0.175952,0.381240,0.015227,0.473018,0.259133,1.000000


In [4]:
# Parece que Total está muy correlacionado con el resto de atributos

min_corr = pokemon[cols].corr().iloc[0,:].min()    # Correlación minima el la fila TOTAL de la matriz corr()

pokemon[cols[1:]].corr() >= min_corr   
# Podemos ver que no existe correlación significativa entre los atributos que suponemos influyen en Total

,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
HP,True,False,False,False,False,False
Attack,False,True,False,False,False,False
Defense,False,False,True,False,False,False
Sp. Atk,False,False,False,True,False,False
Sp. Def,False,False,False,False,True,False
Speed,False,False,False,False,False,True


In [5]:
# Vamos a hacer esta comprobación para todas los tipos de pokemon. para ello nos ayudaremos del chalange 1
types = pd.Series(list(set(pokemon['Type 1'].unique().tolist() + pokemon['Type 2'].unique().tolist())))
types.dropna(inplace=True)


In [21]:
# Voy a explorar como son las correlaciones de total con el resto de atributos, en función del tipo de pokemon
# Solo voy a examinar la columna type1

type_corr_by_total = dict()

for typ in types:    # recorro todos lo tipos únicos de pokemon
    for col in cols:
        df = pokemon[cols][(pokemon['Type 1'] == typ)].corr()       # creo el df de la matriz de correlación sólo para el tipo
        max_corr = df.iloc[0].sort_values(ascending=False)[1]       # obtengo el max ordenando la primera fila (total), y excluyo el 1
        max_index = df.iloc[0].sort_values(ascending=False)[1:].idxmax()
        min_corr = df.iloc[0].min()
        min_index = df.iloc[0].idxmin()        
        type_corr_by_total[typ] ={'max_index':max_index,     # Guardo los datos en un dict
                                 'max_corr':max_corr,
                                 'min_index':min_index,
                                 'min_corr':min_corr}
pd.DataFrame(type_corr_by_total).T        # Paso el dict a un df y lo chequeo

# Se puede ver que la correlación entre total y el resto de atributos. Sobre todo en la tipo 'Flying'
# Pero no siempre la correlación máxima es la misma en función del tipo
# por lo que podemos intuir, que también el tipo de pokemon puede tener correlación con el total

,max_index,max_corr,min_index,min_corr
Electric,Sp. Atk,0.884947,Speed,0.450597
Fairy,Sp. Atk,0.940679,Sp. Def,0.68836
Ghost,Sp. Def,0.675665,Defense,0.529461
Ice,HP,0.717148,Speed,0.509519
Ground,Sp. Atk,0.867559,Speed,0.454975
Rock,Sp. Atk,0.707987,Defense,0.405458
Water,Sp. Atk,0.847236,Speed,0.440602
Flying,Sp. Def,0.982634,Attack,0.921106
Dark,HP,0.766087,Defense,0.601829
Steel,HP,0.845445,Sp. Def,0.539091


## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](../images/one-hot-encoding.png)

In [45]:
# your code here
# no consigo concaternarlas correctamente
pd.concat([pd.get_dummies(pokemon['Type 2']),pd.get_dummies(pokemon['Type 1'])],join='outer',axis=1)

,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,...,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
796,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
797,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
798,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [ ]:
# your code here

# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [ ]:
# your code here